새로운 Jupyter notebook에서 리서치 AI 에이전트를 만들고 커스텀 도구를 부여합니다.
에이전트는 다음 작업을 수행할 수 있어야 합니다:
Wikipedia에서 검색
DuckDuckGo에서 검색
웹사이트의 텍스트를 스크랩하고 추출합니다.
리서치 결과를 .txt 파일에 저장하기
다음 쿼리로 에이전트를 실행합니다:
"Research about the XZ backdoor" 라는 쿼리로 에이전트를 실행하면,
에이전트는 Wikipedia 또는 DuckDuckGo에서 검색을 시도하고,
DuckDuckGo에서 웹사이트를 찾으면 해당 웹사이트에 들어가서 콘텐츠를 추출한 다음,
.txt 파일에 조사 내용을 저장하는 것으로 완료해야 합니다.


In [1]:
from langchain.chat_models import ChatOpenAI

chat_model = ChatOpenAI(temperature=0.1, model_name="gpt-3.5-turbo-1106")

In [4]:
from langchain.tools import BaseTool
from typing import Type
from pydantic import BaseModel, Field
from langchain.utilities import (
    DuckDuckGoSearchAPIWrapper,
    WikipediaAPIWrapper,
)


class QueryArgsSchema(BaseModel):
    query: str = Field(
        description="The query you will research for. Example query: Research about the XZ backdoor"
    )


class DuckduckgoSearchTool(BaseTool):
    name = "DuckduckgoSearchTool"
    description = """
    Use this tool to research about the query in Duckduckgo search engine.
    It takes a query as an argument
    The result of the research will also save as the .txt file in folder name "report"
    When research finish, tell the user about the result and notify the user that research report was created
    """
    args_schema: Type[QueryArgsSchema] = QueryArgsSchema

    def _run(self, query):
        ddg = DuckDuckGoSearchAPIWrapper()
        docs = ddg.run(query)
        file_path = f"./report/{query}.txt"
        with open(file_path, "w", encoding="utf-8") as f:
            f.write(docs)
        return docs


class WikipediaSearchTool(BaseTool):
    name = "WikipediaSearchTool"
    description = """
    Use this tool to research about the query in Wikipedia.
    It takes a query as an argument
    The result of the research will also save as the .txt file in folder name "report"
    When research finish, tell the user about the result and notify the user that research report was created
    """
    args_schema: Type[QueryArgsSchema] = QueryArgsSchema

    def _run(self, query):
        wiki = WikipediaAPIWrapper()
        docs = wiki.run(query)
        file_path = f"./report/{query}.txt"
        with open(file_path, "w", encoding="utf-8") as f:
            f.write(docs)
        return docs


class StringToTextTool(BaseTool):
    name = "StringtoTextTool"

In [6]:
from langchain.agents import initialize_agent, AgentType

agent = initialize_agent(
    llm=chat_model,
    verbose=True,
    agent=AgentType.OPENAI_FUNCTIONS,
    handle_parsing_erros=True,
    tools=[
        WikipediaSearchTool(),
        DuckduckgoSearchTool(),
    ],
)

prompt = "Research about the XZ backdoor, please create report in txt file"

agent.invoke(prompt)



> Entering new AgentExecutor chain...

Invoking: `WikipediaSearchTool` with `{'query': 'XZ backdoor'}`


Page: XZ Utils backdoor
Summary: On 29 March 2024, software developer Andres Freund reported that he had found a maliciously introduced backdoor in the Linux utility xz within the liblzma library in versions 5.6.0 and 5.6.1 released in February 2024.While xz is commonly present in most Linux distributions, the backdoor only targeted Debian- and RPM-based systems running on the x86-64 architecture. At the time of discovery the backdoored version had not yet been widely deployed to production systems, but was present in development versions of major distributions.The backdoor gives an attacker who possesses a specific Ed448 private key remote code execution capabilities on the affected Linux systems. The issue has been assigned a CVSS score of 10.0, the highest possible score.

Page: XZ Utils
Summary: XZ Utils (previously LZMA Utils) is a set of free software command-line lossless d

{'input': 'Research about the XZ backdoor, please create report in txt file',
 'output': "The XZ backdoor refers to a maliciously introduced backdoor in the Linux utility xz within the liblzma library. This backdoor was reported by software developer Andres Freund on 29 March 2024. It targeted Debian- and RPM-based systems running on the x86-64 architecture and allowed an attacker with a specific Ed448 private key to execute remote code on affected Linux systems. The issue has been assigned a CVSS score of 10.0, the highest possible score.\n\nXZ Utils, previously known as LZMA Utils, is a set of free software command-line lossless data compressors, including the programs lzma and xz, for Unix-like operating systems and, from version 5.0 onwards, Microsoft Windows. It uses the Lempel–Ziv–Markov chain algorithm (LZMA) for compression/decompression.\n\nA backdoor is a covert method of bypassing normal authentication or encryption in a computer, product, or embedded device. It is often use